<a href="https://colab.research.google.com/github/youssefkamil/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kazanova/sentiment140

Saving kaggle.json to kaggle.json
 70% 57.0M/80.9M [00:00<00:00, 47.8MB/s]
100% 80.9M/80.9M [00:01<00:00, 79.6MB/s]


In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import nltk
#nltk.download('punkt')


local_zip = '/content/sentiment140.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/data')
zip_ref.close()

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv('/tmp/data/training.1600000.processed.noemoticon.csv', encoding='latin1', names=cols)
#np.random.seed(0)
#index = np.random.randint(low=0, high=1599999, size=1600000)
#df = df.loc[index, ['sentiment', 'text']].reset_index(drop=True)
df = df[['sentiment','text']]
df['sentiment'] = df['sentiment'].replace({4:1})

pat1 = '@[^ ]+'
pat2 = 'http[^ ]+'
pat3 = 'www.[^ ]+'
pat4 = '#[^ ]+'
pat5 = '[0-9]'
pat6 = "(?:s|'s|!+|,|\.|;|:|\(|\)|\"|\?+)?\s|!|!!|!!!|!!!!|%|$" 
pat7 = '(^| ).( |$)'  #remove single char (^| )  ->  Beginning of line or space    .      ->  Any character    ( |$)  ->  Space or End of line

combined_pat = '|'.join((pat1, pat2, pat3, pat4, pat5,pat6,pat7))


clean_tweets = []
for t in df['text']:
    t = t.lower()
    stripped = re.sub(combined_pat, ' ', t)
    a=re.sub(r'\w*\d\w*', '', stripped).strip()
    negations = re.sub("n't", "not", stripped)

    clean_tweets.append(negations)
clean_df = pd.DataFrame(clean_tweets, columns=['text'])
clean_df ['sentiment'] = df['sentiment']

print(clean_df)

                                                      text  sentiment
0            - awww that a bummer  you shoulda got davi...          0
1        i upset that he canot update hi facebook by te...          0
2          i dived many time for the ball managed to sa...          0
3            my whole body feel itchy and like it on fire           0
4          no it not behaving at all i'm mad why am i h...          0
...                                                    ...        ...
1599995  just woke up having no school i the best feeli...          1
1599996  thewdb.com - very cool to hear old walt interv...          1
1599997  are you ready for your mojo makeover ask me fo...          1
1599998  happy   th birthday to my boo of alll time tup...          1
1599999                                     happy                   1

[1600000 rows x 2 columns]


In [ ]:
import collections
pos_df= clean_df[clean_df.sentiment==1]
neg_df=clean_df[clean_df.sentiment==0]
pos_corpus=" "

for text in pos_df['text']:
    pos_corpus+=text
pos_wordlist = pos_corpus.split()
pos_wordlist=collections.Counter(pos_wordlist)
pos_wordlist.most_common(50)

neg_corpus=" "
for text in neg_df['text']:
    neg_corpus+=text

neg_wordlist = neg_corpus.split()
neg_wordlist=collections.Counter(neg_wordlist)
neg_wordlist.most_common(50)
neg_wordlist_df=pd.DataFrame(neg_wordlist.most_common(10000),columns=['words', 'count'])
pos_wordlist_df=pd.DataFrame(pos_wordlist.most_common(10000),columns=['words', 'count'])

neg_wordlist=np.array(neg_wordlist_df['words'])
pos_wordlist=np.array(pos_wordlist_df['words'])



In [ ]:
i=0
x=0
mutual_words=[]
for negword in neg_wordlist:
    for posword in pos_wordlist:
      if negword==posword:
          mutual_words.append(negword)


In [ ]:
with open('/content/drive/My Drive/mutual words.txt', 'w') as f:
    for item in mutual_words:
        f.write("%s\n" % item)

In [ ]:
a=open('/content/drive/My Drive/mutual words.txt','r')
a=a.read()
mutual_words=a.split()
print(len(mutual_words))

19489


In [ ]:
uploaded = files.upload()


Saving mutual words.txt to mutual words.txt


In [ ]:
a=open('/content/mutual words.txt','r')
a=a.read()
mutual_words=a.split()
print(len(mutual_words))

19489


In [ ]:
len(mutual_words)

19489

# **TfIdf**
# Without ngram
# English stop words
# Without max_features
# Without max_df & min_df 
Training Accuracy:  0.79132265625

Testing Accuracy:  0.777728125

In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = TfidfVectorizer(stop_words='english',binary=True)
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  224831


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.79132265625
Testing Accuracy:  0.777728125


(1280000, 224624)

# **CountVectorizer**
# Without ngram
# English stop words
# Without max_features
# Without max_df & min_df 

Training Accuracy:  0.7932046875

Testing Accuracy:  0.77689375

In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = CountVectorizer(stop_words='english',binary=True)
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  224115


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.7932046875
Testing Accuracy:  0.77689375


# **CountVectorizer**
# ngram(1,2)
# English stop words
# Without max_features
# Without max_df & min_df 
Training Accuracy:  0.89824921875

Testing Accuracy:  0.78543125


In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = CountVectorizer(stop_words='english',binary=True,ngram_range=(1,2))
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  3293080


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.894065625
Testing Accuracy:  0.78526875


# **CountVectorizer**
# ngram(1,2)
# English & My stopwords 
# Without max_features
# Without max_df & min_df 
Training Accuracy:  0.87986015625

Testing Accuracy:  0.814346875

In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = CountVectorizer(stop_words=MyStop_words,binary=True,ngram_range=(1,2))
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  3065973


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.87986015625
Testing Accuracy:  0.814346875


In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = CountVectorizer(stop_words=MyStop_words,binary=True,ngram_range=(1,2))
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  3064008


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.87809140625
Testing Accuracy:  0.816084375


# **CountVectorizer**
# ngram(1,2)
# English & My stopwords 
#  max_features =200,000
#  max_df=0.4  
# min_df=1 
Training Accuracy:  0.846559375

Testing Accuracy:  0.817303125

In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as ESW 
mutual_words=frozenset(mutual_words)
MyStop_words=set()
MyStop_words.add(mutual_words)
MyStop_words.add(ESW)

cv = CountVectorizer(stop_words=MyStop_words,binary=True,ngram_range=(1,2),max_features=200000,max_df=0.4,min_df=1)
cv.fit(x_train)
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

vocabulary :  200000


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.846321875
Testing Accuracy:  0.817978125


In [ ]:
cv.vocabulary_

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize
from nltk.corpus import stopwords
#You will have to download the set of punkts & stop words the first time only
import nltk

Using TensorFlow backend.


In [ ]:
#Apply NN Model on count vectorizer
model_1 = Sequential()

model_1.add(Dense( units=32, activation='relu', input_dim=x_train_cv.shape[1]))
model_1.add(Dense( units=16, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile( optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_1.summary()
history1=model_1.fit(x_train_cv, y_train, epochs=5, batch_size=256, validation_data=(x_test_cv,y_test))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                6400032   
_________________________________________________________________
dense_14 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 6,400,577
Trainable params: 6,400,577
Non-trainable params: 0
_________________________________________________________________
Train on 1280000 samples, validate on 320000 samples
Epoch 1/5
1280000/1280000 [==============================] - 1554s 1ms/step - loss: 0.4132 - accuracy: 0.8131 - val_loss: 0.3909 - val_accuracy: 0.8241
Epoch 2/5
1280000/1280000 [==============================] - 1534s 1ms/step - loss: 0.3209 - accuracy: 0.8596 - val_loss: 0.4053 - val_accuracy: 0.8209
Epoch 3/5
1280000

KeyboardInterrupt: ignored

In [ ]:
#Apply NN Model on count vectorizer
model_1 = Sequential()

model_1.add(Dense( units=128, activation='relu', input_dim=x_train_cv.shape[1]))
model_1.add(Dense( units=128, activation='relu'))
model_1.add(Dense( units=128, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile( optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model_1.summary())
history2=model_1.fit(x_train_cv, y_train, epochs=5, batch_size=256, validation_data=(x_test_cv,y_test))


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 128)               25600128  
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 25,633,281
Trainable params: 25,633,281
Non-trainable params: 0
_________________________________________________________________
None
Train on 1280000 samples, validate on 320000 samples
Epoch 1/5
1280000/1280000 [==============================] - 1685s 1ms/step - loss: 0.4099 - accuracy: 0.8135 - val_loss: 0.3877 - val_accuracy: 0.8251
Epoch 2/5
1280000/1280000 [===

In [ ]:
#Apply NN Model on count vectorizer
model_1 = Sequential()

model_1.add(Dense( units=128, activation='relu', input_dim=x_train_cv.shape[1]))
model_1.add(Dropout(0.2))
model_1.add(Dense( units=128, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense( units=128, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile( optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model_1.summary())
history3=model_1.fit(x_train_cv, y_train, epochs=5, batch_size=256, validation_data=(x_test_cv,y_test))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               25600128  
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

KeyboardInterrupt: ignored

In [ ]:
#Apply NN Model on count vectorizer
model_1 = Sequential()

model_1.add(Dense( units=512, activation='relu', input_dim=x_train_cv.shape[1]))
model_1.add(Dropout(0.2))
model_1.add(Dense( units=256, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense( units=128, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense( units=64, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile( optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print(model_1.summary())
history4=model_1.fit(x_train_cv, y_train, epochs=3, batch_size=256, validation_data=(x_test_cv,y_test))


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               102400512 
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)               

KeyboardInterrupt: ignored

In [ ]:
from nltk.stem import PorterStemmer
a="troubling troubling troubling"
lan=PorterStemmer()
lan.stem(a)

'troubling troubling troubl'

In [ ]:
x = clean_df['text']
y = clean_df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


cv = CountVectorizer(stop_words=MyStop_words,binary=True,ngram_range=(1,2),max_df=0.4,min_df=1,max_features=200000)
#cv.fit(x_train)



In [ ]:
a=[]
for w in cv.get_feature_names():
    a.append(lan.stem(w))

In [ ]:
cv.fit(a)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.4, max_features=200000, min_df=1,
                ngram_range=(1, 2), preprocessor=None,
                stop_words={frozenset({'a', 'about', 'above', 'across', 'after',
                                       'afterwards', 'again', 'against', 'all',
                                       'almost', 'alone', 'along', 'already',
                                       '...
                                       'an', 'and', 'another', 'any', 'anyhow',
                                       'anyone', 'anything', 'anyway',
                                       'anywhere', ...}),
                            frozenset({'!', '!!', '!!!', '!!!!', '!!!!!',
                                       '!!!!!!', '!!!!!!!', '!!!!!!!!', '!!!i',
                                       '!!i', '!?', '!i', "!i'm", '!is',
  

In [ ]:
print("vocabulary : ",len(cv.vocabulary_))
x_train_cv = cv.transform(x_train)
x_train_cv.toarray
x_test_cv = cv.transform(x_test) 


vocabulary :  200000


In [ ]:

lr = LogisticRegression()
lr.fit(x_train_cv, y_train)

y_pred_train = lr.predict(x_train_cv)
print('Training Accuracy: ', accuracy_score(y_pred_train, y_train))

y_pred = lr.predict(x_test_cv)
print('Testing Accuracy: ', accuracy_score(y_pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Training Accuracy:  0.80952734375
Testing Accuracy:  0.799234375
